In [1]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import sqlite3
import mysql.connector as cnt
import random
# import setuptools.dist
import undetected_chromedriver as uc
import warnings
warnings.filterwarnings('ignore')

In [2]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 0'''.format(table=table)
    s = c.execute(sql)
    # r = s.description
    column_names = [description[0] for description in c.description]
    return column_names

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(1)
            db.commit()
            time.sleep(2)
    c.close()
    db.close()
    return True


def createMainTable(dbname:str) -> bool:
    sql = '''
    drop table if exists TelePigu
    '''
    db = sqlite3.connect(dbname)
    c = db.cursor()
    c.execute(sql)
    sql = '''create table if not exists TelePigu
    (
        nuoroda text,
        firstInfo text
    )
    '''
    c.execute(sql)
    c.close()
    db.close()
    return True
    pass

def writeMany(lst, table, dbname) -> bool:
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''insert into {table} values ({n})'''
    q = ['?' for i in range(0,len(lst))]
    q_ = ', '.join(q)
    r_ = sql.format(table=table, n=q_)
    # print('::PROVIDED::', lst, len(lst), '::SQL COMMAND::', r_, len(q), sep='\n')
    c.execute(r_, lst)
    db.commit()
    c.close()
    db.close()
    return True


In [3]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
url = 'https://pigu.lt/lt/televizoriai?page=1'
# url = 'https://pigu.lt/lt/saldytuvai/'
driver.get(url)
time.sleep(5)
source = driver.page_source


driver.close()
print('Baigta!!!')

Baigta!!!


In [4]:
bs = BeautifulSoup(source, 'html.parser')
p_nuorodos = []

linkai = bs.find_all('div', {'class':'c-product-card__title'})
for linkas in linkai:
    link = linkas.find('a')['href']
    # print(link)
    p_nuorodos.append(link)

print(len(p_nuorodos))
print(p_nuorodos)


max_page_number = bs.find('a', {'class':'pageSlot-7'}).text
print(max_page_number)



59
['https://pigu.lt/lt/televizoriai/televizorius-xiaomi-tv-a-2025-32?id=89580770', 'https://pigu.lt/lt/televizoriai/televizorius-samsung-qe43q60dauxxh?id=81244960', 'https://pigu.lt/lt/televizoriai/televizorius-xiaomi-qled-tv-a-pro-2025-43?id=84030920', 'https://pigu.lt/lt/televizoriai/televizorius-xiaomi-qled-tv-a-pro-2025-55?id=84030915', 'https://pigu.lt/lt/televizoriai/televizorius-tcl-32s5400a?id=70577291', 'https://pigu.lt/lt/televizoriai/televizorius-lg-32lq63006la?id=52346469', 'https://pigu.lt/lt/televizoriai/televizorius-samsung-ue43du7172uxxh?id=81245065', 'https://pigu.lt/lt/televizoriai/televizorius-estar-led32a2t2?id=61267048', 'https://pigu.lt/lt/televizoriai/televizorius-xiaomi-a-pro-43-108-cm-smart?id=78324768', 'https://pigu.lt/lt/televizoriai/televizorius-lg-oled65c41la?id=85501520', 'https://pigu.lt/lt/televizoriai/televizorius-samsung-ue55du7172uxxh?id=81245055', 'https://pigu.lt/lt/televizoriai/televizorius-estar-led24d5t2?id=36284943', 'https://pigu.lt/lt/televi

In [7]:
for nuoroda in p_nuorodos[1:2]:
    print(nuoroda)
    opcijos = Options()
    opcijos.add_argument('--incognito')
    opcijos.add_argument('--ignore-certificate-errors')
    opcijos.add_argument('--ignore-ssl-errors')
    driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
    url = nuoroda
    driver.get(url)
    time.sleep(5)
    source = driver.page_source


    driver.quit()

https://pigu.lt/lt/televizoriai/televizorius-samsung-qe43q60dauxxh?id=81244960


In [8]:
bs = BeautifulSoup(source, 'html.parser')
firstInfo = bs.find('div', {'class':'c-product__info'}).find('h1', {'class':'c-product__name'}).text.strip()
print(firstInfo)

secondInfo = bs.find('div', {'class':'c-accordion-item__title'}).text.replace('Prekės aprašymas:', '').strip()
print(secondInfo)

# x = eval(price['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
# dict(x)
# x['meta']['sell_price']

kaina = bs.find('div', {'class':'c-product__media'})
price = eval(kaina['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
kaina2 = price['dataLayerItem'].get('price', 'nenurodyta')
print(kaina2)

# if 'price' in price['dataLayerItem']:
#     kaina2 = 'nera'
# else:
#     kaina2 = price['dataLayerItem']['price']
# print(kaina2)


# kaina = bs.find('div', {'class':'c-price h-price--xx-large h-price--new'}).text.strip()
# print(kaina)

specifikacija_keys = []
specifikacija_values = []
specifikacija = bs.find('div', {'class':'h-row h-pl--8 h-pr--8'}).find_all('tr')
for trow in specifikacija:
    td = trow.find_all('td')
    key = td[0].text.strip()
    value = td[1].text.strip()
    specifikacija_keys.append(key)
    specifikacija_values.append(value)

print(len(specifikacija_keys), specifikacija_keys)
print(len(specifikacija_values), specifikacija_values)




Televizorius Samsung QE43Q60DAUXXH, 43" (~109 cm)
Samsung QE43Q60DAUXXH
350
49 ['Prekės ID:', 'Kategorija:', 'Gamintojo kodas:', 'Prekės pakuočių kiekis:', 'Pakuotės išmatavimai ir svoris (1):', 'Prekės ženklas:', 'Modelis:', 'Energijos efektyvumo klasė:', 'Galia:', 'Metinis energijos suvartojimas:', 'Energijos suvartojimas budėjimo rėžime:', 'Ekrano tipas:', 'Rūšis:', 'Smart TV:', 'Lenktas (Curved):', 'TV imtuvai:', 'Svoris:', 'Svoris su stovu:', 'Vaizdo atkūrimo dažnis:', 'Kontrastas:', 'Ryškumas:', 'Garsiakalbio galia (RMS):', 'Jungties tipas:', 'USB jungčių skaičius:', 'Funkcijos:', 'Smart TV funkcijos:', 'Specialiosios funkcijos:', 'WiFi:', 'Bluetooth:', 'Ambilight:', 'Valdymas balsu:', 'Vertikalusis matymo kampas( ° ):', 'Horizontalusis matymo kampas:', 'Spalva:', 'VESA tvirtinimo prie sienos standartas:', 'Kita informacija:', 'Palaikomos technologijos:', 'HDMI jungčių skaičius:', 'Aukštis be stovo:', 'Plotis be stovo:', 'Gylis be stovo:', 'Aukštis su stovu:', 'Plotis su stovu:',

In [ ]:
# createMainTable('VarlePigu.db')

True

In [10]:
# Dronai Pigu scraper

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
url = 'https://pigu.lt/lt/televizoriai?page=1'
driver.get(url)
time.sleep(5)
source = driver.page_source

bs = BeautifulSoup(source, 'html.parser')
max_page_number = bs.find('a', {'class':'pageSlot-7'}).text
last_page = int(max_page_number)

counts = 1
psl = 1
# iki_kokio_psl = 2
# while psl < iki_kokio_psl:
while psl < last_page + 1:
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    p_nuorodos = []
    linkai = bs.find_all('div', {'class':'c-product-card__title'})
    for linkas in linkai:
        link = linkas.find('a')['href']
        # print(link)
        p_nuorodos.append(link)
        
    for nuoroda in p_nuorodos:
        print(psl, nuoroda)
        url = nuoroda
        driver.get(url)
        time.sleep(random.randint(5,10))
        # time.sleep(4)
        source = driver.page_source
        
        cols_i_have = getColNamesList(table='TelePigu', dbname='VarlePigu.db')
        
        duomenys = {}
        for col in cols_i_have:
            duomenys[col] = None
            
        duomenys['nuoroda'] = nuoroda
        
        bs = BeautifulSoup(source, 'html.parser')
        firstInfo = bs.find('div', {'class':'c-product__info'}).find('h1', {'class':'c-product__name'}).text.strip()
        duomenys['firstInfo'] = firstInfo

        secondInfo = bs.find('div', {'class':'c-accordion-item__title'}).text.replace('Prekės aprašymas:', '').strip()
        duomenys['secondInfo'] = secondInfo

        # kaina = bs.find('div', {'class':'c-price h-price--xx-large h-price--new'}).text.strip()
        # print(kaina)

        kaina = bs.find('div', {'class':'c-product__media'})
        price = eval(kaina['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
        kaina2 = price['dataLayerItem'].get('price', 'nenurodyta')
        duomenys['kaina'] = kaina2

        specifikacija_keys = []
        specifikacija_values = []
        specifikacija = bs.find('div', {'class':'h-row h-pl--8 h-pr--8'}).find_all('tr')
        for trow in specifikacija:
            td = trow.find_all('td')
            key = td[0].text.strip()
            value = td[1].text.strip()
            specifikacija_keys.append(key)
            specifikacija_values.append(value)

        for k, v in zip(specifikacija_keys, specifikacija_values):
            duomenys[k] = v
            
        if kaina2 == 'nenurodyta':
            continue
        else:
            cols_i_need = list(duomenys.keys())
        
            data = tuple(duomenys.values())
            
            cols_i_have = getColNamesList(table='TelePigu', dbname='VarlePigu.db')
            
            alterTableAddColumn(cols_i_need, cols_i_have, 'TelePigu', 'VarlePigu.db')
            
            writeMany(data, 'TelePigu', 'VarlePigu.db')
            print(f'Įrašytas įrašas Nr {counts}, puslapis {psl}')
            counts = counts +1
            
    print(f'Įrašytas puslapis {psl}')
    psl = psl +1
    # if psl < iki_kokio_psl:
    if psl <= last_page:
        next_nuoroda = f'https://pigu.lt/lt/televizoriai?page={psl}'
        driver.get(next_nuoroda)
        time.sleep(5)
        print(f'kitas puslapis: {psl}')
        
driver.quit()
print('Baigta!!!')


1 https://pigu.lt/lt/televizoriai/televizorius-xiaomi-tv-a-2025-32?id=89580770
Įrašytas įrašas Nr 1, puslapis 1
1 https://pigu.lt/lt/televizoriai/televizorius-samsung-qe43q60dauxxh?id=81244960
Įrašytas įrašas Nr 2, puslapis 1
1 https://pigu.lt/lt/televizoriai/televizorius-xiaomi-qled-tv-a-pro-2025-43?id=84030920
Įrašytas įrašas Nr 3, puslapis 1
1 https://pigu.lt/lt/televizoriai/televizorius-xiaomi-qled-tv-a-pro-2025-55?id=84030915
Įrašytas įrašas Nr 4, puslapis 1
1 https://pigu.lt/lt/televizoriai/televizorius-tcl-32s5400a?id=70577291
Įrašytas įrašas Nr 5, puslapis 1
1 https://pigu.lt/lt/televizoriai/televizorius-lg-32lq63006la?id=52346469
Įrašytas įrašas Nr 6, puslapis 1
1 https://pigu.lt/lt/televizoriai/televizorius-samsung-ue43du7172uxxh?id=81245065
Įrašytas įrašas Nr 7, puslapis 1
1 https://pigu.lt/lt/televizoriai/televizorius-estar-led32a2t2?id=61267048
Įrašytas įrašas Nr 8, puslapis 1
1 https://pigu.lt/lt/televizoriai/televizorius-xiaomi-a-pro-43-108-cm-smart?id=78324768
Įrašytas 

WebDriverException: Message: unknown error: net::ERR_CONNECTION_CLOSED
  (Session info: chrome=131.0.6778.86)
Stacktrace:
	GetHandleVerifier [0x00D533E3+25059]
	(No symbol) [0x00CDCDE4]
	(No symbol) [0x00BBBEC3]
	(No symbol) [0x00BB9CF4]
	(No symbol) [0x00BAD978]
	(No symbol) [0x00BAF270]
	(No symbol) [0x00BADBC7]
	(No symbol) [0x00BAD567]
	(No symbol) [0x00BAD481]
	(No symbol) [0x00BAB91B]
	(No symbol) [0x00BABE6B]
	(No symbol) [0x00BBE8BE]
	(No symbol) [0x00C3BF47]
	(No symbol) [0x00C21EFC]
	(No symbol) [0x00C3B51E]
	(No symbol) [0x00C21C96]
	(No symbol) [0x00BF3FAC]
	(No symbol) [0x00BF4F3D]
	GetHandleVerifier [0x01045543+3113795]
	GetHandleVerifier [0x0105A20A+3198986]
	GetHandleVerifier [0x010529E2+3168226]
	GetHandleVerifier [0x00DF3250+680016]
	(No symbol) [0x00CE572D]
	(No symbol) [0x00CE29D8]
	(No symbol) [0x00CE2B75]
	(No symbol) [0x00CD57D0]
	BaseThreadInitThunk [0x7694FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778B809E+286]
	RtlGetAppContainerNamedObjectPath [0x778B806E